In [1]:
import pandas as pd

import numpy as np
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

import datetime

plt.style.use('fivethirtyeight')

In [2]:
dataset = pd.read_csv('DataForClustering_AllData_2.csv')
dataset


,Date,XLF,XLK,XLI,HY,XLY,XLU,XLP,SPY,VBMFX,FR3Mts,FR6Mts,FR12Mts
0,12/1/1998,19.039398,32.625000,24.562500,6.44,26.125000,30.234375,27.156250,123.312500,10.27,0.04,0.11,0.19
1,1/1/1999,19.369415,37.812500,24.296875,6.52,27.468750,29.484375,26.796875,127.656250,10.30,0.04,0.04,0.09
2,2/1/1999,19.674046,34.062500,24.515625,6.46,27.296875,28.671875,26.515625,123.562500,10.07,0.05,0.07,0.11
3,3/1/1999,20.257921,36.593750,24.937500,6.57,28.593750,26.687500,26.437500,128.375000,10.06,0.07,0.00,0.17
4,4/1/1999,21.679529,36.812500,28.687500,6.80,29.343750,29.265625,25.515625,133.250000,10.05,0.00,0.03,0.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,8/1/2018,28.330000,75.599998,77.099998,3.44,116.949997,53.470001,53.779999,290.309998,10.44,-0.05,-0.04,0.01
237,9/1/2018,27.580000,75.330002,78.400002,3.44,117.220001,52.650002,53.930000,290.720001,10.36,-0.14,-0.03,0.02
238,10/1/2018,26.280001,69.300003,69.879997,3.37,105.379997,53.689999,55.009998,270.630005,10.26,0.00,0.09,0.12
239,11/1/2018,26.969999,67.940002,72.540001,3.31,107.989998,55.590000,56.259998,275.649994,10.29,0.01,0.00,0.14


In [3]:
dataset['Class'] = np.where(dataset['FR12Mts'] <0, 1, 0)
dataset.tail()

,Date,XLF,XLK,XLI,HY,XLY,XLU,XLP,SPY,VBMFX,FR3Mts,FR6Mts,FR12Mts,Class
236,8/1/2018,28.330000,75.599998,77.099998,3.44,116.949997,53.470001,53.779999,290.309998,10.44,-0.05,-0.04,0.01,0
237,9/1/2018,27.580000,75.330002,78.400002,3.44,117.220001,52.650002,53.930000,290.720001,10.36,-0.14,-0.03,0.02,0
238,10/1/2018,26.280001,69.300003,69.879997,3.37,105.379997,53.689999,55.009998,270.630005,10.26,0.00,0.09,0.12,0
239,11/1/2018,26.969999,67.940002,72.540001,3.31,107.989998,55.590000,56.259998,275.649994,10.29,0.01,0.00,0.14,0
240,12/1/2018,23.820000,61.980000,64.410004,3.22,99.010002,52.919998,50.779999,249.919998,10.45,0.13,0.17,0.29,0


In [4]:
X = dataset.drop(['Date','Class','FR3Mts','FR6Mts','FR12Mts'], 1)
y = dataset['Class']
# X_train = X[0:181]
# y_train = y[0:181]
# X_test = X[182:]
# y_test = y[182:]

In [5]:


from sklearn import metrics
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:

# random forest with random undersampling for imbalanced classification
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.ensemble import BalancedRandomForestClassifier
# generate dataset
# X = dataset.drop(['Date','Target_FR12Mts','Class'], 1)
# y = dataset['Class']
# define model
classifier = BalancedRandomForestClassifier(max_depth = 100,n_estimators=1000,class_weight='balanced_subsample')
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)


In [7]:
# Actual class predictions
rf_predictions = classifier.predict(X_test)
# Probabilities for each class
rf_probs = classifier.predict_proba(X_test)[:, 1]

In [8]:
from sklearn.metrics import roc_auc_score

# Calculate roc auc
roc_value = roc_auc_score(y_test, rf_probs)
roc_value

0.9356521739130435

In [9]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score = accuracy_score(y_test, y_pred)
print(accuracy_score)

[[43  7]
 [ 6 17]]
0.821917808219178


In [10]:
features = list(X_train.columns)
importances = classifier.feature_importances_

fi = pd.DataFrame({'feature': features,
                   'importance': classifier.feature_importances_}).\
                    sort_values('importance', ascending = False)
fi

,feature,importance
3,HY,0.225730
8,VBMFX,0.176000
6,XLP,0.120092
1,XLK,0.117764
4,XLY,0.087006
0,XLF,0.083670
7,SPY,0.072292
2,XLI,0.058756
5,XLU,0.058689


In [62]:

train_rf_predictions = classifier.predict(X_train)
train_rf_probs = classifier.predict_proba(X_train)[:, 1]

rf_predictions = classifier.predict(X_test)
rf_probs =classifier.predict_proba(X_test)[:, 1]

In [63]:
X_test

,XLF,XLK,XLI,HY,XLY,XLU,XLP,SPY,VBMFX
24,23.964256,31.312500,31.250000,5.18,25.593750,33.187500,28.562500,131.187500,9.96
6,21.197197,40.484375,29.812500,6.56,29.781250,30.640625,25.593750,137.000000,9.81
222,24.670000,54.720001,68.110001,3.53,89.629997,51.959999,54.939999,241.800003,10.77
208,18.935825,42.130001,56.160000,3.31,79.199997,48.419998,52.299999,206.330002,10.92
236,28.330000,75.599998,77.099998,3.44,116.949997,53.470001,53.779999,290.309998,10.44
...,...,...,...,...,...,...,...,...,...
192,24.950001,41.349998,57.490002,3.60,72.150002,47.970001,48.490002,205.539993,10.87
104,27.416735,26.040001,39.419998,5.44,37.230000,38.599998,26.900000,147.589996,9.95
109,23.671812,23.250000,37.000000,5.03,32.580002,39.220001,27.200001,137.369995,10.30
234,26.590000,69.470001,71.629997,3.41,109.300003,51.959999,51.529999,271.279999,10.43


In [64]:
print(rf_predictions)
print(rf_probs)

[1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 1 1 1 0 0 1 1 1 0 1 0 1 1 1 0 0
 0 0 0 1 1 1 0 0 1 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 1 0 0]
[0.83473729 0.40926818 0.24326724 0.39398032 0.71346959 0.04917937
 0.35221972 0.21275403 0.3300858  0.75440603 0.47927303 0.37698352
 0.24900748 0.4567171  0.45755206 0.57767821 0.12079927 0.42410287
 0.66681616 0.38207957 0.46391534 0.55087063 0.80643739 0.71681346
 0.06032001 0.14511274 0.51399082 0.6070117  0.51662956 0.34258061
 0.68639687 0.38705596 0.70099606 0.56023471 0.53531092 0.46286204
 0.4172846  0.32917937 0.48972682 0.45714909 0.5018587  0.76558698
 0.77241266 0.46373169 0.26328453 0.61383307 0.32854506 0.78227629
 0.05064904 0.84860106 0.24423106 0.29614772 0.41033586 0.36117035
 0.56403354 0.4585745  0.37255489 0.0875626  0.78058046 0.408541
 0.27957676 0.46498917 0.23209502 0.13856452 0.47204975 0.56100566
 0.44401587 0.33557787 0.3721651  0.70215327 0.52839776 0.4800022
 0.47578338]


In [13]:
# X_test['Prob'] = rf_probs.tolist()
# X_test

# frames = [dropped_values, X_test]

# result = pd.concat(frames, axis=1, sort=False)
# result

# result.to_csv(r'RF_DOWN_Results_42.csv')